In [10]:
import pandas as pd

In [11]:
# data_path = "../data/data_investeringer.xlsx"
data_path = "../data/investeringer_datagrundlag.xlsx"
df = pd.read_excel(data_path)

In [12]:
df

,Kommune,ISIN kode,Værdipapirets navn,Udsteder,Markedsværdi (DKK),Type
0,Rødovre,AEA000801018,Abu Dhabi Islamic Bank PJSC,Abu Dhabi Islamic Bank PJSC,4.420592e+03,Aktie
1,Holstebro,AEA002001013,Aldar Properties,Aldar Properties,4.891252e+04,Aktie
2,Fanø,AEA002001013,Aldar Properties PJSC,United Arab Emirates,5.660577e+01,Aktie
3,Fanø,AEE000301011,Emaar Properties PJSC,United Arab Emirates,3.962404e+02,Aktie
4,Fanø,AEE000401019,Emirates Telecommunications Gr,United Arab Emirates,3.962404e+02,Aktie
...,...,...,...,...,...,...
140967,Kalundborg,ZAG000125972,REPUBLIC OF SOUTH AFRICA 8.875% 28.02.2035,Sydafrika,2.644064e+04,Obligation
140968,Vejen,ZAG000125980,REPUBLIC OF SOUTH AFRICA 9% 31.01.2040,Republic of South Africa Government Bond,8.144535e+04,Obligation
140969,Kalundborg,ZAG000125980,REPUBLIC OF SOUTH AFRICA 9% 31.01.2040,Sydafrika,9.191644e+03,Obligation
140970,Kalundborg,ZAG000125980,REPUBLIC OF SOUTH AFRICA 9% 31.01.2040,Sydafrika,1.379931e+04,Obligation


In [13]:
# Filter the rows where "Kommune" is missing
missing_kommune_rows = df[df['Kommune'].isna()]

# Display the rows
print(missing_kommune_rows)

df = df[df['Kommune'].notna()]

       Kommune ISIN kode Værdipapirets navn Udsteder  Markedsværdi (DKK) Type
140971     NaN       NaN                NaN      NaN        6.858794e+10  NaN


In [17]:
data_path = "../data/full_data.xlsx"
full_data = pd.read_excel(data_path)
full_data.head()

,Kommune,ISIN kode,Værdipapirets navn,Udsteder,Markedsværdi (DKK),Type,ISIN,Selskab,Årsag til eksklusion,Selskab_normalized,Matched Udsteder,Matched Værdipapirets navn,Kommuner,Eksklusionsårsager,OBS_Type,Priority,Problematisk ifølge:,Sortlistet,Problemkategori
0,Rødovre,AEA000801018,Abu Dhabi Islamic Bank PJSC,Abu Dhabi Islamic Bank PJSC,4420.591824,Aktie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
1,Holstebro,AEA002001013,Aldar Properties,Aldar Properties,48912.517111,Aktie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
2,Fanø,AEA002001013,Aldar Properties PJSC,United Arab Emirates,56.605770,Aktie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
3,Fanø,AEE000301011,Emaar Properties PJSC,United Arab Emirates,396.240390,Aktie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
4,Fanø,AEE000401019,Emirates Telecommunications Gr,United Arab Emirates,396.240390,Aktie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN


In [19]:
# Group the data by 'Kommune' and check if there are any rows with a value in 'Priority'
grouped = full_data.groupby('Kommune')['Priority'].apply(lambda x: x.notna().any() and x.isin([1, 2, 3]).any())

# List of municipalities with at least one row with a priority value of 1, 2, or 3
municipalities_with_priority = grouped[grouped == True].index.tolist()

# List of municipalities with no rows having priority values
municipalities_without_priority = grouped[grouped == False].index.tolist()

# Display the results
print("Municipalities with at least one row with Priority (1, 2, or 3):")
print(municipalities_with_priority)

print("\nMunicipalities with no Priority values (1, 2, or 3):")
print(municipalities_without_priority)


Municipalities with at least one row with Priority (1, 2, or 3):
['Aabenraa', 'Aalborg', 'Aarhus', 'Assens', 'Ballerup', 'Billund', 'Bornholm', 'Brøndby', 'Brønderslev', 'Egedal', 'Esbjerg', 'Faaborg-Midtfyn', 'Fanø', 'Favrskov', 'Faxe', 'Fredensborg', 'Fredericia', 'Frederiksberg', 'Frederikssund', 'Furesø', 'Gladsaxe', 'Greve', 'Gribskov', 'Guldborgsund', 'Haderslev', 'Hedensted', 'Helsingør', 'Herning', 'Hillerød', 'Holbæk', 'Holstebro', 'Horsens', 'Hvidovre', 'Høje Taastrup', 'Hørsholm', 'Ikast-Brande', 'Jammerbugt', 'Kalundborg', 'Kerteminde', 'Kolding', 'København', 'Køge', 'Lejre', 'Lemvig', 'Lolland', 'Lyngby-Taarbæk', 'Mariagerfjord', 'Middelfart', 'Morsø', 'Norddjurs', 'Nordfyn', 'Nyborg', 'Næstved', 'Odense', 'Randers', 'Rebild', 'Region Hovedstaden', 'Region Midtjylland', 'Region Nordjylland', 'Region Sjælland', 'Ringkøbing-Skjern', 'Roskilde', 'Rudersdal', 'Rødovre', 'Silkeborg', 'Skanderborg', 'Skive', 'Slagelse', 'Solrød', 'Sorø', 'Struer', 'Svendborg', 'Syddjurs', 'This

In [20]:
# Check that the lists are completely different
intersection = set(municipalities_with_priority).intersection(municipalities_without_priority)

# Display the lists and check for overlap
if not intersection:
    print("The lists are completely distinct.")
else:
    print("The following municipalities appear in both lists:")
    print(intersection)

The lists are completely distinct.


In [15]:
print(sum(full_data['Markedsværdi (DKK)']))
print(sum(df['Markedsværdi (DKK)']))

68584013870.05557
68587938726.39119


In [16]:
import numpy as np

# Replace '-' with NaN (Fjerner dem, hvor der ikke er værdi. Det er fx to fra Odense)
df['Markedsværdi (DKK)'] = df['Markedsværdi (DKK)'].replace('-', np.nan)

# Remove any potential commas, spaces, or other non-numeric characters
df['Markedsværdi (DKK)'] = df['Markedsværdi (DKK)'].replace({',': '', ' ': ''}, regex=True)

# Convert the column to float
df['Markedsværdi (DKK)'] = df['Markedsværdi (DKK)'].astype(float)


C:\Users\Anja\AppData\Local\Temp\ipykernel_9376\1433036786.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Markedsværdi (DKK)'] = df['Markedsværdi (DKK)'].replace('-', np.nan)
C:\Users\Anja\AppData\Local\Temp\ipykernel_9376\1433036786.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Markedsværdi (DKK)'] = df['Markedsværdi (DKK)'].replace({',': '', ' ': ''}, regex=True)
C:\Users\Anja\AppData\Local\Temp\ipykernel_9376\1433036786.py:10: SettingWithCopyWarning: 
A value is trying to be set on a c

In [6]:
df.head()

,Kommune,ISIN kode,Værdipapirets navn,Udsteder,Markedsværdi (DKK),Type
0,Rødovre,AEA000801018,Abu Dhabi Islamic Bank PJSC,Abu Dhabi Islamic Bank PJSC,4420.591824,Aktie
1,Holstebro,AEA002001013,Aldar Properties,Aldar Properties,48912.517111,Aktie
2,Fanø,AEA002001013,Aldar Properties PJSC,United Arab Emirates,56.605770,Aktie
3,Fanø,AEE000301011,Emaar Properties PJSC,United Arab Emirates,396.240390,Aktie
4,Fanø,AEE000401019,Emirates Telecommunications Gr,United Arab Emirates,396.240390,Aktie


In [33]:
import pandas as pd

# Create the function to fill missing 'Type' based on the majority for each 'ISIN kode'
def fill_missing_type(df, min_rows=5, agree_threshold=0.8):
    def fill_type_for_group(group):
        # Count the missing values in 'Type' for this group
        missing_count = group['Type'].isna().sum()
        #print(f"ISIN kode: {group.name}, Missing 'Type' values: {missing_count}")
        
        # Get the count of each type in the group, excluding missing values
        type_counts = group['Type'].value_counts()
        
        # If there are no valid types in the group, skip this group
        if type_counts.empty:
            return group
        
        total_rows = len(group)
        most_common_type, most_common_count = type_counts.idxmax(), type_counts.max()
        
        # Check the condition: at least min_rows, and agreement should meet the threshold
        if total_rows >= min_rows and most_common_count / total_rows >= agree_threshold:
            # If conditions met, fill missing 'Type' with the most common type
            group['Type'] = group['Type'].fillna(most_common_type)
        
        return group

    # Group by 'ISIN kode' and apply the function to each group
    df = df.groupby('ISIN kode').apply(fill_type_for_group)

    return df

# Print the number of missing values in 'Type' before applying the function
missing_before = df['Type'].isna().sum()
print(f"Missing 'Type' values before: {missing_before}")

# Apply the function to fill missing 'Type' values
filled_df = fill_missing_type(df, min_rows=5, agree_threshold=0.80)

# Print the number of missing values in 'Type' after applying the function
missing_after = filled_df['Type'].isna().sum()
print(f"Missing 'Type' values after: {missing_after}")

filled_df.head()


Missing 'Type' values before: 7279
Missing 'Type' values after: 2098


C:\Users\Anja\AppData\Local\Temp\ipykernel_12048\2663692735.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('ISIN kode').apply(fill_type_for_group)


Kommune     ISIN kode              Værdipapirets navn  \
ISIN kode                                                                 
AEA000801018 0    Rødovre  AEA000801018     Abu Dhabi Islamic Bank PJSC   
AEA002001013 1  Holstebro  AEA002001013                Aldar Properties   
             2       Fanø  AEA002001013           Aldar Properties PJSC   
AEE000301011 3       Fanø  AEE000301011           Emaar Properties PJSC   
AEE000401019 4       Fanø  AEE000401019  Emirates Telecommunications Gr   

                                   Udsteder  Markedsværdi (DKK)   Type  
ISIN kode                                                               
AEA000801018 0  Abu Dhabi Islamic Bank PJSC         4420.591824  Aktie  
AEA002001013 1             Aldar Properties        48912.517111  Aktie  
             2         United Arab Emirates           56.605770  Aktie  
AEE000301011 3         United Arab Emirates          396.240390  Aktie  
AEE000401019 4         United Arab Emirates          396.240390  Aktie

In [15]:
df_lav = df[df['Markedsværdi (DKK)'] < 100]
df_lav = df_lav[df_lav['Markedsværdi (DKK)'] >= 0]
df_lav

,Kommune,ISIN kode,Værdipapirets navn,Udsteder,Markedsværdi (DKK),Type
386,Sorø,DE000LEG1268,LEG IMMOBILIEN SE-SCRIP,LEG Immobilien SE,30.853651,Aktie
1027,Ikast-Brande,DE000LEG1268,LEG IMMOBILIEN SE-SCRIP,LEG Immobilien SE,85.332755,Aktie
1565,Ikast-Brande,FR001400OP17,EssilorLuxottica SA Rettighed 2024,EssilorLuxottica SA Rettighed 2024,0.000000,Rettighed
1566,Ikast-Brande,NL00150023I0,Ferrovial SE Rettighed 2024,Ferrovial SE Rettighed 2024,0.000000,Rettighed
1567,Ikast-Brande,DE000LEG1268,LEG Immobilien SE Rettighed 2024,LEG Immobilien SE Rettighed 2024,0.000000,Rettighed
...,...,...,...,...,...,...
128662,Region Syddanmark,DK0009535478,3.00% Nykredit Realkredit AS 2053,Nykredit Realkredit AS,0.001860,Obligation
128696,Region Syddanmark,DK0009356388,5.00% Jyske Realkredit A/S 2025,Jyske Realkredit A/S,67.855568,Obligation
128706,Region Syddanmark,DK0009292120,2.50% Realkredit Danmark A/S 2036,Realkredit Danmark A/S,6.539539,Obligation
128717,Region Syddanmark,DK0009252769,7.00% Realkredit Danmark A/S 2024,Realkredit Danmark A/S,8.624031,Obligation


In [16]:
sum(df_lav['Markedsværdi (DKK)'])

37758.40218330926

In [5]:
df.value_counts('Kommune')

Kommune
Guldborgsund       6441
Faaborg-Midtfyn    3556
Kalundborg         3519
Mariagerfjord      3449
Hørsholm           3397
                   ... 
Gentofte              5
Vordingborg           5
Ærø                   4
Dragør                3
Albertslund           1
Name: count, Length: 102, dtype: int64

In [6]:
text = df[df['Kommune'] == 'Guldborgsund'].to_string(index=False)

In [9]:
!pip install tiktoken

   ---------------------------------------- 0.0/799.3 kB ? eta -:--:--
   -------------------------- ------------- 524.3/799.3 kB 8.2 MB/s eta 0:00:01
   ---------------------------------------- 799.3/799.3 kB 3.4 MB/s eta 0:00:00


In [12]:
import tiktoken
import pandas as pd

# Function to convert dataframe to text and count tokens
def dataframe_to_text(df):
    """
    This function converts a dataframe into a text format.
    You can modify it based on how you want to present the data.
    """
    text = df.to_string(index=False)  # Convert DataFrame to string (or JSON if needed)
    return text

# Function to count tokens using OpenAI's tokenizer (tiktoken)
def count_tokens(text, model="gpt-4"):
    """
    Counts the number of tokens in a given text based on the OpenAI model.
    """
    # Load the encoding for the specified model (gpt-3.5-turbo, gpt-4, etc.)
    encoding = tiktoken.encoding_for_model(model)
    
    # Encode the text into tokens
    tokens = encoding.encode(text)
    
    # Return the number of tokens
    return len(tokens)

# Convert dataframe to text
df_text = dataframe_to_text(df['Markedsværdi (DKK)'][df['Kommune'] == 'Guldborgsund'])

# Count tokens
num_tokens = count_tokens(df_text, model="gpt-4")

# print(f"DataFrame text:\n{df_text}")
print(f"\nNumber of tokens: {num_tokens}")


Number of tokens: 51527


### Danwatch-check

In [2]:
import pandas as pd

In [3]:
data_path = "../data/datagrundlag_FN.xlsx"
df = pd.read_excel(data_path)

In [16]:
df_fn = df[df['Problematisk ifølge:'] == 'FN']
sum(round(df_fn['Markedsværdi (DKK)'],5))

17042332.33889

In [11]:
len(df_fn)

355

In [2]:
import pandas as pd
from sqlalchemy import create_engine
import os 
import base64
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding


# Decrypt data using AES-CBC mode
def aes_decrypt(encrypted_data, key):
    encrypted_data = base64.b64decode(encrypted_data.encode())  # Decode Base64 to bytes
    iv = encrypted_data[:16]  # Extract the first 16 bytes as the IV
    ciphertext = encrypted_data[16:]

    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()

    decrypted_padded_data = decryptor.update(ciphertext) + decryptor.finalize()

    # Remove padding
    unpadder = padding.PKCS7(128).unpadder()
    decrypted_data = unpadder.update(decrypted_padded_data) + unpadder.finalize()

    return decrypted_data.decode()

# Function to decrypt specified columns of the DataFrame using AES-CBC
def decrypt_dataframe(df, key, col_list):
    df_decrypted = df.copy()  # Create a copy of the DataFrame

    for col in col_list:
        if pd.api.types.is_string_dtype(df_decrypted[col]):
            # Decrypt string columns
            df_decrypted[col] = df_decrypted[col].apply(lambda x: aes_decrypt(x, key))
        else:
            # Convert to the original data type after decryption
            df_decrypted[col] = df_decrypted[col].apply(lambda x: aes_decrypt(str(x), key))

    return df_decrypted

# Only encrypt the necessary columns
col_list = ["Kommune", "ISIN kode", "Værdipapirets navn"]

# Fetch the base64 encoded key from environment variables
encoded_key = os.getenv("ENCRYPTION_KEY")

# Decode the key back to bytes for AES use
encryption_key = base64.b64decode(encoded_key)


# Create an SQLite engine
engine = create_engine("sqlite:///../src/investerings_database_encrypted_new.db")

# --- Optional: Decrypt the DataFrame for verification ---
df_retrieved = pd.read_sql("SELECT * FROM kommunale_regioner_investeringer", engine)
df_decrypted = decrypt_dataframe(df_retrieved, encryption_key, col_list)
print("Decrypted DataFrame:")
print(df_decrypted.head())


Decrypted DataFrame:
     Kommune     ISIN kode              Værdipapirets navn  \
0    Rødovre  AEA000801018     Abu Dhabi Islamic Bank PJSC   
1  Holstebro  AEA002001013                Aldar Properties   
2       Fanø  AEA002001013           Aldar Properties PJSC   
3       Fanø  AEE000301011           Emaar Properties PJSC   
4       Fanø  AEE000401019  Emirates Telecommunications Gr   

                      Udsteder  Markedsværdi (DKK)   Type  \
0  Abu Dhabi Islamic Bank PJSC         4420.591824  Aktie   
1             Aldar Properties        48912.517111  Aktie   
2         United Arab Emirates           56.605770  Aktie   
3         United Arab Emirates          396.240390  Aktie   
4         United Arab Emirates          396.240390  Aktie   

  Årsag til eksklusion OBS_Type  Priority Problematisk ifølge:  Sortlistet  \
0                 None     None       NaN                 None           0   
1                 None     None       NaN                 None           0   
2    

In [3]:
# Step 1: Filter for rows where 'Priority' is 2 or 3
priority_2_or_3_df = df_decrypted[df_decrypted['Priority'].isin([2, 3])]

# Step 2: Get unique 'Kommune' values where 'Priority' is 2 or 3
kommuner_with_priority_2_or_3 = priority_2_or_3_df['Kommune'].unique()

# Step 3: Get all unique 'Kommune' values in the DataFrame
all_kommuner = df_decrypted['Kommune'].unique()

# Step 4: Find 'Kommune' values that don't have any 'Priority' of 2 or 3
kommuner_without_priority_2_or_3 = list(set(all_kommuner) - set(kommuner_with_priority_2_or_3))

# Print the results
print("Kommunes with Priority 2 or 3:", kommuner_with_priority_2_or_3)
print("Kommunes without Priority 2 or 3:", kommuner_without_priority_2_or_3)


Kommunes with Priority 2 or 3: ['Rødovre' 'Skive' 'Svendborg' 'Esbjerg' 'Mariagerfjord' 'Rudersdal'
 'Fanø' 'Aarhus' 'Kalundborg' 'Vejen' 'Norddjurs' 'Aalborg' 'Fredericia'
 'Greve' 'Herning' 'Hillerød' 'Høje Taastrup' 'Hørsholm' 'Kolding'
 'Lemvig' 'Næstved' 'Randers' 'Varde' 'Aabenraa' 'Assens' 'Billund'
 'Bornholm' 'Egedal' 'Favrskov' 'Frederiksberg' 'Gribskov' 'Guldborgsund'
 'Holstebro' 'Horsens' 'Hvidovre' 'Lejre' 'Lyngby-Taarbæk' 'Nordfyn'
 'Nyborg' 'Rebild' 'Roskilde' 'Skanderborg' 'Solrød' 'Thisted' 'Tønder'
 'Vallensbæk' 'Viborg' 'Faaborg-Midtfyn' 'Hedensted' 'Helsingør'
 'Ikast-Brande' 'Middelfart' 'Odense' 'Slagelse' 'Syddjurs' 'Ballerup'
 'Brøndby' 'Brønderslev' 'Fredensborg' 'Kerteminde' 'Køge' 'Lolland'
 'Morsø' 'Region Midtjylland' 'Region Nordjylland' 'Region Sjælland'
 'Silkeborg' 'Sorø' 'Struer' 'Ringkøbing-Skjern' 'Region Hovedstaden'
 'Holbæk' 'Furesø' 'København' 'Vesthimmerland' 'Jammerbugt' 'Vejle'
 'Faxe' 'Haderslev']
Kommunes without Priority 2 or 3: ['Odsherr

In [4]:
df_decrypted.columns

Index(['Kommune', 'ISIN kode', 'Værdipapirets navn', 'Udsteder',
       'Markedsværdi (DKK)', 'Type', 'Årsag til eksklusion', 'OBS_Type',
       'Priority', 'Problematisk ifølge:', 'Sortlistet', 'Eksklusionsårsager',
       'Problemkategori'],
      dtype='object')

In [3]:
import pandas as pd

# Assuming df_decrypted is your existing dataframe
# Step 1: Split the 'Problemkategori' column by ';' and expand them into separate rows
df_expanded = df_decrypted.assign(
    Problemkategori=df_decrypted['Problemkategori'].str.split(';')
).explode('Problemkategori')

# Step 2: Strip any extra spaces in 'Problemkategori'
df_expanded['Problemkategori'] = df_expanded['Problemkategori'].str.strip()

# Step 3: Grouping by 'Kommune' and 'Problemkategori', and aggregating
df_grouped = df_expanded.groupby(['Kommune', 'Problemkategori']).agg(
    Antal=('Problemkategori', 'size'),  # Counting occurrences
    Markedsværdi_sum=('Markedsværdi (DKK)', 'sum')  # Summing the values
).reset_index()

# Step 4: Rounding the 'Markedsværdi_sum' to 2 decimal places
df_grouped['Markedsværdi_sum'] = df_grouped['Markedsværdi_sum'].round(2)


# Saving the result to a CSV file
df_grouped.to_csv('investeringer_problemkategori.csv', index=False)

print(df_grouped)


       Kommune                Problemkategori  Antal  Markedsværdi_sum
0     Aabenraa  Aktiviteter i besatte områder     11         157788.22
1     Aabenraa                          Andet     40        1717288.05
2     Aabenraa        Arbejdstagerrettigheder     18        1047575.52
3     Aabenraa                  Biodiversitet      1          18328.53
4     Aabenraa                  EU-sanktioner      2          41458.33
...        ...                            ...    ...               ...
1005    Viborg                          Klima      4         423342.69
1006    Viborg          Kontroversielle våben      1           1039.09
1007    Viborg            Menneskerettigheder     10         486576.41
1008    Viborg                          Tobak      2          27666.18
1009    Viborg               Usunde fødevarer     14        2091690.31

[1010 rows x 4 columns]


In [12]:
import pandas as pd

# Step 0: Filter the dataframe to keep only rows where 'Priority' is 2 or 3
df_filtered = df_decrypted[df_decrypted['Priority'].isin([2, 3])]

# Step 1: Split the 'Problemkategori' column by ';' and expand them into separate rows
df_expanded = df_filtered.assign(
    Problemkategori=df_filtered['Problemkategori'].str.split(';')
).explode('Problemkategori')

# Step 2: Strip any extra spaces in 'Problemkategori'
df_expanded['Problemkategori'] = df_expanded['Problemkategori'].str.strip()

# Step 3: Create a crosstab for 'Kommune' and 'Problemkategori'
df_crosstab = pd.crosstab(df_expanded['Kommune'], df_expanded['Problemkategori'])

# Step 4: Adding a 'Total' column that sums up all non-empty 'Problemkategori' counts
df_crosstab['Total'] = df_crosstab.sum(axis=1)

# Step 5: Get the total 'Markedsværdi (DKK)' sum for each 'Kommune' from the original df_decrypted
df_markedsværdi_sum = df_filtered.groupby('Kommune')['Markedsværdi (DKK)'].sum().reset_index()

df_result = df_crosstab.merge(df_markedsværdi_sum, on='Kommune')

df_result['Markedsværdi (DKK)'] = df_result['Markedsværdi (DKK)'].round(2)

# Step 6: Saving the result to a CSV file
df_result.to_csv('investeringer_problemkategori_kolonner.csv', index=False)

print(df_result)


           Kommune  Aktiviteter i besatte områder  Alkohol  Andet  \
0         Aabenraa                             11        0     40   
1          Aalborg                              2        0     19   
2           Aarhus                              3        0     23   
3           Assens                              7        0     28   
4         Ballerup                              0        0     14   
..             ...                            ...      ...    ...   
74           Varde                              4        0     35   
75           Vejen                              1        5    115   
76           Vejle                              4        0      2   
77  Vesthimmerland                              5        0     41   
78          Viborg                              8        0     13   

    Arbejdstagerrettigheder  Biodiversitet  EU-sanktioner  \
0                        18              1              2   
1                        11              3       